Read cytobands(hg38) from a file, for each sample, count the overlaps in each band than save to db.  

Also conduct filtering, sample with more than 3000 segments are dropped out

In [2]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import copy

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

### Setup

In [ ]:
db = MongoClient()['Rebased']['mecaned']
samples = []
for sam in db.find():
    samples.append(sam)

Load and parse cytobands

In [1]:
# bands is a list of 24 lists, represents 24 chromosomes. Each sub-list stores the cytobands of that chromosome as another list
bands = []
for i in range(24):
    bands.append([])
    

with open('/Users/bogao/DataFiles/Data/genome/cytoBand_hg38.txt','r') as fin:
    for line in fin:
        tokens = line.strip().split('\t')
        if ('_' in tokens[0]) or ('M' in tokens[0]):
            continue
        elif 'X' in tokens[0]:
            chro = 22
        elif 'Y' in tokens[0]:
            chro = 23
        else:
            chro = int(tokens[0][3:])-1

        info = {'start':int(tokens[1]), 'end':int(tokens[2]), 'name':tokens[3], 'note':tokens[4],
                'total_dup':0, 'total_del':0, 'dup_length':0, 'del_length':0, 'dup_count':0, 'del_count':0}
        bands[chro].append(info)

Function

In [3]:
def cytobandOverlap(cytobands, dbin, vtype='normalized'):

    c = 1
    for sam in dbin.find():
        this_bands = copy.deepcopy(cytobands)
        if (vtype in sam.keys()) and (sam[vtype] != None) and (len(sam['segments'])<3000):
            for seg in sam[vtype]:
                if seg['probes'] >4:
                    
                    if seg['value'] >2:
                        val = 2
                    else:
                        val = seg['value']
                    
                    if seg['chro'] == 'X':
                        chro = 22
                    elif seg['chro'] == 'Y':
                        chro = 23
                    else:
                        chro = int(seg['chro'])-1

                    # find all matching bands
                    for b in this_bands[chro]:
                        if (b['start'] < seg['end']) and (b['end'] >= seg['start']):
                            size = min(b['end'],seg['end']) - max(b['start'],seg['start'])

                            # dup
                            if seg['value'] > 0:
                                # update total value
                                b['total_dup'] += size*seg['value']
                                # update total_length
                                b['dup_length'] += size
                                # update count
                                b['dup_count'] +=1
                            # del
                            else:
                                # update total value
                                b['total_del'] += size*seg['value']
                                # update total_length
                                b['del_length'] += size
                                # update count
                                b['del_count'] +=1

                # flat the nested bands
                flat_bands = []
                idx = 1
                for ch in this_bands:
                    if idx == 23:
                        chro = 'X'
                    elif idx == 24:
                        chro = 'Y'
                    else:
                        chro = str(idx)
                    for b in ch:
                        # add chromosom
                        b['chro'] = chro
                        # compute average signals
                        if b['dup_length'] >0:
                            b['ave_dup'] = b['total_dup'] / b['dup_length']
                        else:
                            b['ave_dup'] = 0

                        if b['del_length'] >0:
                            b['ave_del'] = b['total_del'] / b['del_length']
                        else:
                            b['ave_del'] = 0

                        flat_bands.append(b)
                    idx +=1        
            sam['cytobands'] = flat_bands
            dbin.update_one({'sample_id':sam['sample_id']}, {'$set':{'cytobands':flat_bands}}, upsert=True)
            print(c, end='\r')
            c +=1
    return 0

#### Run

In [4]:
cytobandOverlap(bands, MongoClient()['Rebased']['mecaned'])

0